# Welcome to Semantic Kernel with Microsoft Fabric - Getting Started
- This notebook has following examples:
    - install semantic-kernel library in Microsoft Fabric Spark environment
    - Run Skills inline, directly in the code
    - Run Skills from a file in the LakeHouse
    - stream text/chat completions
- Todo: Secrets are hardcoded in this notebook for simplicity for the workshop - to move to Azure Keyvault
- Prerequisites:
    - Access to Azure OpenAI, and values for the endpoint, the key, the deployments
    - Upload Skills to the Lakehouse 

## Get Started

In [1]:
#By adding library in workspace setting, you have already installed semantic-kernel! 
#in case you have not setup in workspace setting, then uncomment below line and run this cell
#
#%pip install semantic-kernel --quiet

%pip show semantic-kernel

StatementMeta(, 71784aaa-d751-4c35-84a3-571d6c48abec, 5, Finished, Available)

Name: semantic-kernel
Version: 0.3.12.dev0
Summary: 
Home-page: 
Author: Microsoft
Author-email: SK-Support@microsoft.com
License: 
Location: /home/trusted-service-user/cluster-env/trident_env/lib/python3.10/site-packages
Requires: aiofiles, numpy, openai, openapi_core, prance, pydantic, python-dotenv, regex
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [2]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai import ChatRequestSettings, CompleteRequestSettings
from semantic_kernel.connectors.ai.open_ai import AzureTextCompletion, AzureChatCompletion

StatementMeta(, 71784aaa-d751-4c35-84a3-571d6c48abec, 6, Finished, Available)

In [7]:
api_key = "*******"
endpoint = "https://*****.openai.azure.com/"
azure_text_service = AzureTextCompletion("text-davinci-003", endpoint, api_key)
azure_chat_service = AzureChatCompletion("gpt-35-turbo", endpoint, api_key)

StatementMeta(, 71784aaa-d751-4c35-84a3-571d6c48abec, 11, Finished, Available)

In [8]:
kernel = sk.Kernel()

StatementMeta(, 71784aaa-d751-4c35-84a3-571d6c48abec, 12, Finished, Available)

In [9]:
request_settings = CompleteRequestSettings(
    max_tokens=512,
    temperature=0.7,
    top_p=1,
    frequency_penalty=0.5,
    presence_penalty=0.5
)

StatementMeta(, 71784aaa-d751-4c35-84a3-571d6c48abec, 13, Finished, Available)

## Example - Stream text completions

In [10]:
prompt = "provide me a list of use cases for OpenAI and Microsoft Fabric"
stream = azure_text_service.complete_stream_async(prompt, request_settings)
async for text in stream:
    print(text, end = "") # end = "" to avoid newlines

StatementMeta(, 71784aaa-d751-4c35-84a3-571d6c48abec, 14, Finished, Available)



1. Using OpenAI to Develop a Chatbot: OpenAI's GPT-3 platform can be used to develop conversational AI applications like chatbots, natural language processing (NLP) systems, and virtual assistants.

2. Creating an Intelligent Virtual Agent with Microsoft Fabric: Microsoft Fabric is a powerful tool for creating intelligent virtual agents that can understand user intent and provide helpful responses. It can be used to build customer service bots, automated customer support systems, and more.

3. Implementing Games with OpenAI: OpenAI provides powerful tools for developing games using deep learning techniques. With its API, developers can create games that are rich in visuals and interactivity while being easy to learn and play.

4. Predictive Analytics with Microsoft Fabric: By leveraging machine learning algorithms, Microsoft Fabric can help businesses gain insight into their customers' behavior and make predictions about trends in the market. This data can be used to improve marketin

In [11]:
prompt = """
Microsoft Fabric is an all-in-one analytics solution for enterprises that covers everything from data movement to data science, Real-Time Analytics, and business intelligence. It offers a comprehensive suite of services, including data lake, data engineering, and data integration, all in one place.
With Fabric, you don't need to piece together different services from multiple vendors. Instead, you can enjoy a highly integrated, end-to-end, and easy-to-use product that is designed to simplify your analytics needs.
The platform is built on a foundation of Software as a Service (SaaS), which takes simplicity and integration to a whole new level.
Microsoft Fabric brings together new and existing components from Power BI, Azure Synapse, and Azure Data Factory into a single integrated environment. These components are then presented in various customized user experiences.

Generate a slogan or sales pitch.
"""

stream = azure_text_service.complete_stream_async(prompt, request_settings)
async for text in stream:
    print(text, end = "")


StatementMeta(, 71784aaa-d751-4c35-84a3-571d6c48abec, 15, Finished, Available)


"Unlock the power of analytics with Microsoft Fabric: the all-in-one solution for data movement, real-time analytics, and business intelligence!"

In [12]:
kernel.add_text_completion_service("text_completion", azure_text_service)

StatementMeta(, 71784aaa-d751-4c35-84a3-571d6c48abec, 16, Finished, Available)

## Example - Semantic Function using Skills inline

In [13]:
# note: using inline Skills
prompt = """
Generate a short funny poem or limerick to explain the given event. Be creative and be funny. Let your imagination run wild.

+++++

Event:{{$input}}
+++++
"""
promptFunction = kernel.create_semantic_function(
        prompt,
        max_tokens=100,
        temperature=0.9,
        top_p=0.0,
        presence_penalty=0.0,
        frequency_penalty=0.0,
    )

input_text = "Technical Workshop"
completions = await promptFunction.invoke_async(input_text)

print(completions)

StatementMeta(, 71784aaa-d751-4c35-84a3-571d6c48abec, 17, Finished, Available)


A technical workshop was held one day
Where techies gathered to learn and play
They discussed and debated
Till their brains were abated
And the workshop was a success, hooray!


## Semantic Function using Skills from Files in Lakehouse

In [14]:
# note: using skills from the samples folder
skills_directory = "/lakehouse/default/Files/sk-samples-skills"

SummarizeFunction = kernel.import_semantic_skill_from_directory(skills_directory, "SummarizeSkill")["Summarize"]


SampleText = """Microsoft Fabric is an all-in-one analytics solution for enterprises that covers everything from data movement to data science, Real-Time Analytics, and business intelligence. It offers a comprehensive suite of services, including data lake, data engineering, and data integration, all in one place.
With Fabric, you don't need to piece together different services from multiple vendors. Instead, you can enjoy a highly integrated, end-to-end, and easy-to-use product that is designed to simplify your analytics needs.
The platform is built on a foundation of Software as a Service (SaaS), which takes simplicity and integration to a whole new level.
Microsoft Fabric brings together new and existing components from Power BI, Azure Synapse, and Azure Data Factory into a single integrated environment. These components are then presented in various customized user experiences.

"""
result = await SummarizeFunction.invoke_async(SampleText)
print(result)

StatementMeta(, 71784aaa-d751-4c35-84a3-571d6c48abec, 18, Finished, Available)


Microsoft Fabric is an all-in-one analytics solution that provides data movement, data science, Real-Time Analytics, and business intelligence services in one place. It is built on a foundation of Software as a Service (SaaS) and integrates components from Power BI, Azure Synapse, and Azure Data Factory into a single environment with customized user experiences.


## Example - Stream Chat compltions

In [15]:
chat_request_settings = ChatRequestSettings(
    max_tokens=150,
    temperature=0.7,
    top_p=1,
    frequency_penalty=0.5,
    presence_penalty=0.5,
)

StatementMeta(, 71784aaa-d751-4c35-84a3-571d6c48abec, 19, Finished, Available)

In [16]:
prompt = "Today is going to be a great day, I can feel it. I'm going to finish all exercies in my technical workshop and ..."
stream = azure_chat_service.complete_chat_stream_async([("user", prompt)], chat_request_settings)
async for text in stream:
    print(text, end = "") # end = "" to avoid newlines

StatementMeta(, 71784aaa-d751-4c35-84a3-571d6c48abec, 20, Finished, Available)

assistant: learn something new that will help me in my career. I'm going to be productive and efficient, and make the most out of every opportunity that comes my way. I'll take breaks when needed to recharge and refocus, but I won't let any distractions get in the way of achieving my goals for the day.

I'll also make sure to connect with my colleagues and build positive relationships with them. Collaboration is key, and working together can bring about innovative ideas and solutions.

At the end of the day, I'll reflect on everything I've accomplished and feel proud of myself for pushing through any challenges that came up. With this positive attitude and mindset, every day has the potential to be a great day.

## End of this Notebook